## Table of Contents

1. [Defining the optimization problem: search space and objective](#1-defining-the-optimization-problem-search-space-and-objective)  
2. [First touch with Optuna for optimization](#2-first-touch-with-optuna-for-optimization)  
3. [Analyzing the optimization results](#3-analyzing-the-optimization-results)  
4. [Setting up baselines with enqueue trials](#4-setting-up-baselines-with-enqueue-trials)



## 1. Defining the optimization problem: search space and objective

## 2. First touch with Optuna for optimization

## 3. Analyzing the optimization results

## 4. Setting up baselines with enqueue trials